In [1]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it
    
from functools import partial
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne

import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine
from ieeg.viz.parula import parula_map

# TODO: hmm fix these utils imports, import the funcs individually. 6/1/25.

import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm
from scipy.stats import ttest_ind

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import copy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup

from src.analysis.config import experiment_conditions

from src.analysis.utils.labeled_array_utils import (
    put_data_in_labeled_array_per_roi_subject,
    remove_nans_from_labeled_array,
    remove_nans_from_all_roi_labeled_arrays,
    concatenate_conditions_by_string,
    get_data_in_time_range
)

from src.analysis.decoding.decoding import (
    concatenate_and_balance_data_for_decoding, 
    get_and_plot_confusion_matrix_for_rois_jim,
    Decoder, 
    windower,
    get_confusion_matrices_for_rois_time_window_decoding_jim,
    compute_accuracies,
    perform_time_perm_cluster_test_for_accuracies,
    plot_accuracies
)

from src.analysis.spec.wavelet_functions import get_uncorrected_wavelets, get_uncorrected_multitaper, get_sig_tfr_differences, plot_mask_pages
from src.analysis.spec.subjects_tfr_objects_functions import load_or_make_subjects_tfr_objects, get_sig_tfr_differences_per_subject, get_sig_tfr_differences_per_roi

from src.analysis.utils.general_utils import (
    make_or_load_subjects_electrodes_to_ROIs_dict,
    get_good_data,
    get_sig_chans_per_subject,
    make_sig_electrodes_per_subject_and_roi_dict,
    calculate_total_electrodes,
    check_sampling_rates
)

import mne.time_frequency
from ieeg.calc.scaling import rescale
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np
from sklearn.model_selection import StratifiedKFold
from typing import Union, List, Sequence, Dict
import doctest

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)



['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\

#### 0. Load data.   
Need a way to load in the frequency information too, not just trials x channels x timepoints. Because I'm going to use frequency as a decoding feature too. For each electrode, for each training set, I think I can just mask the multitaper with the significant clusters and use that as the decoding feature, and then concatenate across electrodes to build the full training matrix.

In [2]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']

# params - these will become input variables once i functionalize this stuff
subjects = ['D0103']
signal_times = [-1.0, 1.5]
acc_trials_only = False
error_trials_only = False
stat_func = partial(ttest_ind, equal_var=False, nan_policy='omit')
p_thresh = 0.2
ignore_adjacency = 1 # ignore the channels dimension for clusters, just find clusters over frequency and time
n_perm = 10
n_jobs = 1
freqs = np.arange(2, 200., 4.)
n_cycles = freqs / 2
return_itc = False
time_bandwidth=10 
spec_method = 'multitaper'
average=False
seed=None
tails=2
n_splits=2
n_repeats=1
random_state=42

# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')
HOME = os.path.expanduser("~")

USER = os.path.basename(HOME)

if os.name == 'nt':  # Windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
elif sys.platform == 'darwin':  # macOS
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
else:  # Linux (cluster)
    # Check if we're on the cluster by looking for /cwork directory
    if os.path.exists(f"/cwork/{USER}"):
        LAB_root = f"/cwork/{USER}"
    else:
        # Fallback for other Linux systems
        LAB_root = os.path.join(HOME, "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

task='GlobalLocal'
conditions = experiment_conditions.stimulus_big_letter_conditions # set this to whichever conditions you're running

stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = list(conditions.keys()) # get the condition names as a list

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [3]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [4]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0103


get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [5]:
# rois_dict = {
#     'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# rois_dict = {
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
# }

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
all_electrodes_per_subject_roi, sig_electrodes_per_subject_roi = make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LAS

In [6]:
print('total sig elecs:', sum(len(sig_chans_per_subject[sub]) for sub in sig_chans_per_subject))

total sig elecs: 82


get number of sig and all electrodes per subject and across subjects

In [7]:
for roi in rois:
    for sub in subjects:
        sig_elecs = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        all_elecs = all_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        print(f"Subject {sub}, ROI {roi}, Num of Sig Electrodes: {len(sig_elecs)}, Num of All Electrodes: {len(all_elecs)}")

Subject D0103, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 11
Subject D0103, ROI occ, Num of Sig Electrodes: 3, Num of All Electrodes: 4


In [8]:
# Example usage:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, all_electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 8
Total number of lpfc electrodes across all subjects: 265
Total number of significant occ electrodes across all subjects: 3
Total number of occ electrodes across all subjects: 73


#### 1. For each electrode, make multitaper using all training trials, for both conditions to be compared (this can only do two conditions, can't do more rn)

TODO: Need to loop over the trials and use some as training set, and have some held out as a test set that isn't used to find clusters. Maybe do this in the stats step though.

In [9]:
subjects_tfr_objects = load_or_make_subjects_tfr_objects(
    layout=layout,
    spec_method=spec_method,
    conditions_save_name=conditions_save_name,
    subjects=subjects,
    conditions=conditions,
    signal_times=signal_times,
    freqs=freqs,
    n_cycles=n_cycles,
    time_bandwidth=time_bandwidth,
    return_itc=return_itc,
    n_jobs=n_jobs,
    average=average,
    acc_trials_only=acc_trials_only,
    error_trials_only=error_trials_only 
)

Found existing TFR data. Loading from: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\multitaper\subjects_tfr_objects\stimulus_big_letter_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_1_subjects_multitaper


#### 2. Find clusters that are significantly different between the two conditions, in the multitaper spectrogram  
do this for all channels per subject, and then also for all channels in an roi across subjects


In [10]:
# # THIS TAKES FOREVER DON'T RUN IT, JUST FOR DEBUGGING SO I CAN PLOT THE SIG CLUSTERS

# # For per-subject analysis (no electrode filtering needed)
# sig_elec_masks_per_subject, sig_elec_pvals_per_subject = get_sig_tfr_differences_per_subject(subjects_tfr_objects=subjects_tfr_objects, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# all_elec_masks_per_subject, all_elec_pvals_per_subject = get_sig_tfr_differences_per_subject(subjects_tfr_objects=subjects_tfr_objects, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# # For per-ROI analysis (with electrode filtering)
# sig_elec_masks_per_roi, sig_elec_pvals_per_roi = get_sig_tfr_differences_per_roi(subjects_tfr_objects=subjects_tfr_objects, electrodes_per_subject_roi=sig_electrodes_per_subject_roi, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# all_elec_masks_per_roi, all_elec_pvals_per_roi = get_sig_tfr_differences_per_roi(subjects_tfr_objects=subjects_tfr_objects, electrodes_per_subject_roi=all_electrodes_per_subject_roi, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

plot these clusters

In [11]:
# # TODO: Update the mean differences to be per subject and per roi, currently copied over from wavelet_differences.
# first_sub = subjects[0]
# first_condition = list(subjects_tfr_objects[first_sub].keys())[0]
# ch_names = subjects_tfr_objects[first_sub][first_condition].ch_names
# times = subjects_tfr_objects[first_sub][first_condition].times
# freqs = subjects_tfr_objects[first_sub][first_condition].freqs

# subjects_tfr_objects_save_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, 'subjects_tfr_objects')

# # Now plot the mask pages:
# for sub in subjects:
#     sig_elecs_mask = sig_elec_masks_per_subject[sub]
#     sig_elecs_mask_pages = plot_mask_pages(sig_elecs_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(sig_elecs_mask_pages):
#         fig_name = f"{sub}_sig_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

#     all_elecs_mask = all_elec_masks_per_subject[sub]
#     all_elecs_mask_pages = plot_mask_pages(all_elecs_mask,
#                         ch_names,
#                         times=times,
#                         freqs=freqs,
#                         channels_per_page=60,
#                         grid_shape=(6, 10),
#                         cmap=parula_map,
#                         title_prefix=f"{sub} ",
#                         log_freq=True,
#                         show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(all_elecs_mask_pages):
#         fig_name = f"{sub}_all_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

# for roi in rois:
#     sig_elecs_roi_mask = sig_elec_masks_per_roi[roi]
#     sig_elecs_roi_mask_pages = plot_mask_pages(sig_elecs_roi_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} {roi} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(sig_elecs_roi_mask_pages):
#         fig_name = f"{sub}_{roi}_sig_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

#     all_elecs_roi_mask = all_elec_masks_per_roi[roi]
#     all_elecs_roi_mask_pages = plot_mask_pages(all_elecs_roi_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} {roi} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(all_elecs_roi_mask_pages):
#         fig_name = f"{sub}_{roi}_all_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)
            
# # # get the mean differences themselves and plot them
# # mean_diff_inc_vs_con = mean_diff(incongruent_spec._data, congruent_spec._data, axis=0)
# # mean_diff_switch_vs_repeat = mean_diff(switch_spec._data, repeat_spec._data, axis=0)

# # # Now, plot the mean differences directly:
# # congruency_mean_diff_pages = plot_mask_pages(
# #     mean_diff_inc_vs_con,
# #     incongruent_spec.ch_names,
# #     times=incongruent_spec.times,
# #     freqs=incongruent_spec.freqs,
# #     grid_shape=(6, 10),
# #     cmap=parula_map,  # play with color maps
# #     title_prefix=f"{sub} Mean Inc-Con Diff: ",
# #     log_freq=True,
# #     show=False
# # )

# # # Save each page as a separate figure file:
# # for i, fig in enumerate(congruency_mean_diff_pages):
# #     if rescaled:
# #         fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
# #     else:
# #         fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
# #     fig_pathname = os.path.join(save_dir, fig_name)
# #     fig.savefig(fig_pathname, bbox_inches='tight')
# #     print("Saved figure:", fig_name)

let's try creating roi labeled arrays and then masking these and decoding using them

make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.


In [22]:
# Example usage:

# choose whether to use sig elecs or all elecs
electrodes = all_electrodes_per_subject_roi # toggle this to sig_electrodes_per_subject_roi if just using sig elecs, or electrodes_per_subject_roi if using all elecs

if electrodes == all_electrodes_per_subject_roi:
    elec_string_to_add_to_filename = 'all_elecs'
elif electrodes == sig_electrodes_per_subject_roi:
    elec_string_to_add_to_filename = 'sig_elecs'
else:
    elec_string_to_add_to_filename = None

roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(
    subjects_tfr_objects,
    condition_names,
    rois,
    subjects,
    electrodes, 
    obs_axs=0,  # Trials dimension (ignoring the conditions dimension for now cuz we iterate over it)
    chans_axs=1,  # Channels dimension
    freq_axs=2,
    time_axs=3,   # Time dimension
    random_state=42  # For reproducibility
)

ROI 'lpfc': Maximum trials per condition:
  Condition 'Stimulus_bigS': 176 trials from subjects ['D0103']
  Condition 'Stimulus_bigH': 183 trials from subjects ['D0103']
in roi lpfc, subject D0103 has 176 trials for condition Stimulus_bigS
in roi lpfc, subject D0103 has 183 trials for condition Stimulus_bigH
Subject D0103, ROI lpfc, LabeledArray shape: (2, 183, 11, 3, 142)
ROI 'occ': Maximum trials per condition:
  Condition 'Stimulus_bigS': 176 trials from subjects ['D0103']
  Condition 'Stimulus_bigH': 183 trials from subjects ['D0103']
in roi occ, subject D0103 has 176 trials for condition Stimulus_bigS
in roi occ, subject D0103 has 183 trials for condition Stimulus_bigH
Subject D0103, ROI occ, LabeledArray shape: (2, 183, 4, 3, 142)


In [ ]:
# move all this into decoding.py once i get it working
def balance_and_decode_with_tfr_masking(
    X_train_raw, y_train, X_test_raw, y_test,
    train_indices, test_indices, 
    roi_labeled_array, condition_names, obs_axs, chans_axs,
    stat_func, p_thresh, n_perm,
    Decoder, cats, balance_method, 
    n_bootstraps, random_state,
    ignore_adjacency=1, seed=42, tails=2
):
    """
    Balance data and decode with TFR cluster masking.
    
    Parameters:
    -----------
    X_train_raw : np.ndarray
        Raw training data (trials, channels, freqs, times)
    y_train : np.ndarray
        Training labels
    X_test_raw : np.ndarray
        Raw test data
    y_test : np.ndarray
        Test labels
    train_indices : np.ndarray
        Indices of training trials in the original roi_labeled_array
    test_indices : np.ndarray
        Indices of test trials
    roi_labeled_array : LabeledArray
        The full roi labeled array for extracting channel info
    condition_names : list
        List of condition names to compare
    obs_axs : int
        Axis of the roi_labeled_array that contains trial labels
    chans_axs : int
        Axis of the roi_labeled_array that contains channel labels
    stat_func : callable
        Statistical function to use for permutation cluster test
    p_thresh : float
        P-value threshold for permutation cluster test
    n_perm : int
        Number of permutations for permutation cluster test
    Decoder : class
        Decoder class to use for decoding
    cats : list
        List of category names
    balance_method : str
        Method to balance data ('subsample' or 'pad_with_nans')
    n_bootstraps : int
        Number of bootstraps for decoding
    random_state : int
        Random seed for reproducibility
    ignore_adjacency : int
        Number of adjacent channels to ignore when computing clusters
    seed : int
        Random seed for permutation cluster test
    tails : int
        Number of tails for permutation cluster test
    """
    
    # Get channel labels from the labeled array as a list
    channel_labels = roi_labeled_array.labels[chans_axs+1] # add 1 to the chans axs because of the conditions axis that got addd
    
    # Step 1: Create training-only TFR masks
    channel_masks = compute_tfr_masks_from_roi_labeled_array(
        roi_labeled_array, train_indices, condition_names, 
        obs_axs, chans_axs,
        stat_func, p_thresh, n_perm, 
        ignore_adjacency, seed, tails
    )
    
    # Step 2: Apply masks and flatten (now passing channel_labels)
    X_train = apply_tfr_masks_and_flatten(X_train_raw, channel_masks, channel_labels)
    X_test = apply_tfr_masks_and_flatten(X_test_raw, channel_masks, channel_labels)
    
    # Step 3: Balance data - wait hold on, is this not already balanced using concatenate_and_balance_data_for_decoding
    if balance_method == 'subsample':
        # Remove NaN trials
        valid_train = ~np.isnan(X_train).any(axis=1)
        X_train = X_train[valid_train]
        y_train = y_train[valid_train]
        
        # Subsample to balance classes
        X_train, y_train = subsample_to_balance(X_train, y_train, random_state) # this function doesn't exist
    
    # Step 4: Decode
    decoder = Decoder(cats, 0.80, n_splits=1, n_repeats=1, oversample=(balance_method==balance_method))
    decoder.fit(X_train, y_train)
    preds = decoder.predict(X_test)
    
    return confusion_matrix(y_test, preds)


def compute_tfr_masks_from_roi_labeled_array(
    roi_labeled_array, train_indices, condition_names,
    obs_axs, chans_axs, stat_func, p_thresh, n_perm, 
    ignore_adjacency=1, seed=42, tails=2
):
    """
    Compute significant TFR clusters using only training trials from roi_labeled_array.
    
    Returns:
    --------
    channel_masks : dict
        {channel_label: mask_array} where mask is (n_freqs, n_times)
    """
    channel_masks = {}
    
    # Get channel labels from the labeled array
    channel_labels = roi_labeled_array.labels[chans_axs+1]
    
    # Split training data by condition
    train_data_by_condition = {}
    for cond in condition_names:  # hmm the stats only work for two conditions, so just do two conditions for now. Can expand to >2 conditions in the future, would just need to do ANOVA i think instead of time perm cluster.
        # Extract training trials for this condition
        cond_data = roi_labeled_array[cond]  # Shape: (trials, channels, freqs, times). Test this!
        cond_train_data = np.take(cond_data, train_indices, axis=obs_axs) # TODO: keep going through this code 4:45 on 8/1 - huh? this is a 4d array, check what train_indices is. Should grab along the trials axis. Maybe do np.take(train_indices, axis=obs_axs) to be safe.
        train_data_by_condition[cond] = cond_train_data
    
    # For each channel, compute significant clusters
    n_channels = len(channel_labels)
    for ch_idx in range(n_channels):
        ch_label = channel_labels[ch_idx]
        
        # Get data for this channel across conditions
        cond_data_this_chan = {}
        for cond in condition_names:
            cond_data_this_chan[cond] = np.take(train_data_by_condition[cond], ch_idx, axis=chans_axs)
        
        if cond_data_this_chan.keys() != 2:
            raise ValueError("For now, just doing perm test instead of ANOVA, so this will only work for two conditions, sorry!")
        
        # TODO: Once I implement ANOVA, don't hard code cond1 and cond2 data, just use the cond_data_this_chan dict and feed it into some get_sig_tfr_differences function that implements ANOVA instead of time perm cluster. 
        cond1_data = cond_data_this_chan[condition_names[0]]
        cond2_data = cond_data_this_chan[condition_names[1]]
        # Run time perm cluster (TODO: implement ANOVA too as another option for when i'm decoding more than two conditions)
        if len(cond1_data) > 0 and len(cond2_data) > 0:
            mask, _ = get_sig_tfr_differences(
                cond1_data, cond2_data,
                stat_func=stat_func,
                p_thresh=p_thresh,
                n_perm=n_perm,
                axis=obs_axs,  # trials axis
                ignore_adjacency=ignore_adjacency,
                seed=seed,
                tails=tails
            )
            channel_masks[ch_label] = mask
        else:
            # No data for comparison
            channel_masks[ch_label] = np.zeros(
                (cond1_data.shape[1], cond1_data.shape[2]), dtype=bool
            )
            raise ValueError(f"cond 1 has {len(cond1_data)} trials and cond 2 has {len(cond2_data)} trials. Can't decode! ")
    
    return channel_masks


def apply_tfr_masks_and_flatten(data, obs_axs, chans_axs, channel_masks, channel_labels):
    """
    Apply channel-specific TFR masks and flatten to create feature matrix.
    
    Parameters:
    -----------
    data : np.ndarray
        Shape: (n_trials, n_channels, n_freqs, n_times)
    obs_axs : int
        Axis of the data that contains trial labels
    chans_axs : int
        Axis of the data that contains channel labels
    channel_masks : dict
        {channel_label: mask_array} where channel_label is like "D0103-LPFC1"
    channel_labels : list
        List of channel labels in the same order as data's channel dimension
    
    Returns:
    --------
    feature_matrix : np.ndarray
        Shape: (n_trials, n_features)
    """
    n_trials, n_channels = data.shape[obs_axs], data.shape[chans_axs]
    feature_vectors = []
    
    # Iterate through each channel
    for ch_idx in range(n_channels):
        # Extract this channel's data for all trials
        # Shape changes from (n_trials, n_channels, n_freqs, n_times) 
        # to (n_trials, n_freqs, n_times)
        channel_data = np.take(data, ch_idx, chans_axs)
        
        # Get the label for this channel index
        ch_label = channel_labels[ch_idx]  # e.g., "D0103-LPFC1"
        
        # Check if we have a mask for this channel
        if ch_label in channel_masks:
            # Get the boolean mask (n_freqs, n_times)
            mask = channel_masks[ch_label]
            
            # Apply mask to each trial's data
            # channel_data[:, mask] selects only the True positions in mask, because mask is n freqs x n times
            # This reduces (n_trials, n_freqs, n_times) to (n_trials, n_significant_features)
            masked_features = channel_data[:, mask]
            
            # Note: reshape(n_trials, -1) is redundant here since it's already 2D
            # masked_features is already (n_trials, n_features)
        else:
            # No mask for this channel - use all time-frequency points
            # Flatten from (n_trials, n_freqs, n_times) to (n_trials, n_freqs*n_times)
            masked_features = channel_data.reshape(n_trials, -1)
        
        # Add this channel's features to our list
        feature_vectors.append(masked_features)
    
    # Concatenate all channels' features horizontally
    # If channel 1 has 50 features and channel 2 has 30 features,
    # result will have 80 features total
    feature_matrix = np.concatenate(feature_vectors, axis=chans_axs)
    return feature_matrix

def get_and_plot_confusion_matrix_for_rois_tfr_cluster(
    roi_labeled_arrays, rois, condition_comparison, strings_to_find, 
    save_dir, stat_func, p_thresh=0.05, n_perm=100,
    time_interval_name=None, other_string_to_add=None, 
    n_splits=5, n_repeats=5, obs_axs=0, 
    balance_method='subsample', random_state=42
):
    """
    Modified version that uses TFR cluster masking.
    """
    confusion_matrices = {}
    
    for roi in rois:
        print(f"Processing ROI: {roi}")
        
        # Get data and labels
        concatenated_data, labels, cats = concatenate_and_balance_data_for_decoding(
            roi_labeled_arrays, roi, strings_to_find, obs_axs, balance_method, random_state
        )
        
        # Set up cross-validation
        all_cms = []
        
        for repeat in range(n_repeats):
            repeat_seed = random_state + repeat * 1000
            skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=repeat_seed)
            
            fold_cms = []
            
            for fold_idx, (train_indices, test_indices) in enumerate(skf.split(concatenated_data, labels)):
                print(f"  Repeat {repeat+1}/{n_repeats}, Fold {fold_idx+1}/{n_splits}")
                
                # Get train/test data
                X_train_raw = concatenated_data[train_indices]
                X_test_raw = concatenated_data[test_indices]
                y_train = labels[train_indices]
                y_test = labels[test_indices]
                
                # Balance and decode with TFR masking
                cm = balance_and_decode_with_tfr_masking(
                    X_train_raw, y_train, X_test_raw, y_test,
                    train_indices, test_indices,
                    roi_labeled_arrays[roi], list(roi_labeled_arrays[roi].keys()), obs_axs, chans_axs,
                    stat_func, p_thresh, n_perm,
                    Decoder, cats, balance_method, 
                    1, repeat_seed + fold_idx
                )
                
                fold_cms.append(cm)
            
            # Sum across folds
            repeat_cm = np.sum(fold_cms, axis=0)
            all_cms.append(repeat_cm)
        
        # Average across repeats
        final_cm = np.mean(all_cms, axis=0)
        confusion_matrices[roi] = final_cm
        
        # Plot (your existing plotting code)
        # ...
    
    return confusion_matrices

below code is older approaches

decoding functions that concatenate the significant cluster mask for each electrode in an roi, made using training data, and test on just the concatenated significant cluster masks in an roi on test data  
commenting this out cuz i don't understand it, will build my own code...

In [ ]:
# def collect_roi_data(subjects_tfr_objects, electrodes_per_subject_roi, roi, subjects, condition_names):
#     """
#     Collect all data for a specific ROI across subjects and conditions.
    
#     Returns:
#     --------
#     all_data : np.array
#         Shape: (n_trials, n_channels, n_freqs, n_time)
#     trial_labels : list
#         Labels for each trial (e.g., "D0057_r25_trial0")
#     condition_labels : list
#         Condition name for each trial
#     channel_labels : list
#         Channel names for this ROI
#     freq_labels : list
#         Frequency values
#     time_labels : list
#         Time values
#     """
#     all_data_list = []
#     trial_labels = []
#     condition_labels = []
#     channel_labels = None
#     freq_labels = None
#     time_labels = None
    
#     for sub in subjects:
#         roi_electrodes = electrodes_per_subject_roi.get(roi, {}).get(sub, [])
#         if not roi_electrodes:
#             continue
            
#         for cond_name in condition_names:
#             tfr = subjects_tfr_objects[sub][cond_name]
            
#             # Get indices of ROI electrodes
#             electrode_indices = []
#             for elec in roi_electrodes:
#                 if elec in tfr.ch_names:
#                     electrode_indices.append(tfr.ch_names.index(elec))
            
#             if not electrode_indices:
#                 continue
            
#             # Extract data for these electrodes
#             data = tfr.data[:, electrode_indices, :, :]  # (trials, channels, freqs, time)
            
#             # Add to lists
#             for trial_idx in range(len(data)):
#                 all_data_list.append(data[trial_idx])
#                 trial_labels.append(f"{sub}_{cond_name}_trial{trial_idx}")
#                 condition_labels.append(cond_name)
            
#             # Set labels if not set
#             if channel_labels is None:
#                 channel_labels = [tfr.ch_names[i] for i in electrode_indices]
#             if freq_labels is None:
#                 freq_labels = tfr.freqs.tolist()
#             if time_labels is None:
#                 time_labels = tfr.times.tolist()
    
#     all_data = np.array(all_data_list) if all_data_list else np.array([])
    
#     return all_data, trial_labels, condition_labels, channel_labels, freq_labels, time_labels


# def create_train_tfr_subset(subjects_tfr_objects, train_trials, subjects, condition_names):
#     """
#     Create a subset of TFR objects containing only training trials.
    
#     Returns:
#     --------
#     train_tfrs : dict
#         Deep copy of TFR objects filtered to only training trials
#     """
#     train_tfrs = {}
    
#     for sub in subjects:
#         train_tfrs[sub] = {}
#         for cond in condition_names:
#             # Find training trials for this subject/condition
#             train_trial_indices = []
#             for trial_label in train_trials:
#                 if trial_label.startswith(f"{sub}_{cond}_"):
#                     trial_num = int(trial_label.split('_trial')[1])
#                     train_trial_indices.append(trial_num)
            
#             if train_trial_indices:
#                 train_tfrs[sub][cond] = subjects_tfr_objects[sub][cond][train_trial_indices]
#             else:
#                 # Create empty TFR
#                 train_tfrs[sub][cond] = subjects_tfr_objects[sub][cond][:0]
    
#     return train_tfrs


# def create_feature_mask(train_masks, electrodes_per_subject_roi, roi, subjects, 
#                        channel_labels, freq_labels, time_labels, tfr):
#     """
#     Create a feature mask based on significant clusters from training data.
    
#     Returns:
#     --------
#     feature_mask : np.array
#         Boolean mask of shape (n_channels, n_freqs, n_times)
#     """
#     feature_mask = np.zeros((len(channel_labels), len(freq_labels), len(time_labels)), dtype=bool)
    
#     for sub in subjects:
#         if sub not in train_masks:
#             continue
            
#         roi_electrodes = electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        
#         for elec_name in roi_electrodes:
#             if elec_name in tfr.ch_names and elec_name in channel_labels:
#                 orig_elec_idx = tfr.ch_names.index(elec_name)
#                 la_elec_idx = channel_labels.index(elec_name)
                
#                 if orig_elec_idx < train_masks[sub].shape[0]:
#                     feature_mask[la_elec_idx, :, :] = train_masks[sub][orig_elec_idx, :, :]
    
#     return feature_mask


# def extract_and_clean_features(train_data, test_data, feature_mask):
#     """
#     Extract features using mask and handle NaN values.
    
#     Returns:
#     --------
#     X_train : np.array
#         Training features with NaNs imputed
#     X_test : np.array
#         Test features with NaNs imputed
#     """
#     # Reshape to (n_trials, n_features_total)
#     # MODIFICATION: Use .data to get the raw NumPy array
#     X_train = np.asarray(train_data).reshape(len(train_data), -1)
#     X_test = np.asarray(test_data).reshape(len(test_data), -1)
    
#     # Apply feature mask
#     mask_flat = feature_mask.flatten()
#     X_train = X_train[:, mask_flat]
#     X_test = X_test[:, mask_flat]
    
#     # Handle NaN values (Ahhhh this is wrong, replace either with what james does or do mixup)
#     col_mean = np.zeros(X_train.shape[1])
#     if np.any(np.isnan(X_train)):
#         col_mean = np.nanmean(X_train, axis=0)
#         nan_mask = np.isnan(X_train)
#         X_train[nan_mask] = np.take(col_mean, np.where(nan_mask)[1])
    
#     if np.any(np.isnan(X_test)):
#         nan_mask = np.isnan(X_test)
#         # Use the mean calculated from the training data to fill NaNs in the test data
#         X_test[nan_mask] = np.take(col_mean, np.where(nan_mask)[1])
    
#     return X_train, X_test

# def balance_training_data(X_train, y_train, random_state):
#     """
#     Balance training data by subsampling to the smallest class.
    
#     Returns:
#     --------
#     X_train_balanced : np.array
#         Balanced training features
#     y_train_balanced : np.array
#         Balanced training labels
#     """
#     unique_labels, counts = np.unique(y_train, return_counts=True)
#     min_trials = np.min(counts)
    
#     balanced_indices = []
#     rng = np.random.RandomState(random_state)
    
#     for label_val in unique_labels:
#         label_indices = np.where(y_train == label_val)[0]
#         subsampled = rng.choice(label_indices, min_trials, replace=False)
#         balanced_indices.extend(subsampled)
    
#     X_train_balanced = X_train[balanced_indices]
#     y_train_balanced = y_train[balanced_indices]
    
#     return X_train_balanced, y_train_balanced

# def run_single_cv_fold(fold_data):
#     """
#     Run a single cross-validation fold using PCA-LDA classifier.
    
#     Parameters:
#     -----------
#     fold_data : dict
#         Dictionary containing all necessary data for one fold
        
#     Returns:
#     --------
#     confusion_matrix : np.array
#         Confusion matrix for this fold
#     """
#     # Extract parameters
#     train_trials = fold_data['train_trials']
#     test_trials = fold_data['test_trials']
#     train_data = fold_data['train_data']
#     test_data = fold_data['test_data']
#     y_train = fold_data['y_train']
#     y_test = fold_data['y_test']
#     subjects_tfr_objects = fold_data['subjects_tfr_objects']
#     subjects = fold_data['subjects']
#     condition_names = fold_data['condition_names']
#     stat_func = fold_data['stat_func']
#     p_thresh = fold_data['p_thresh']
#     n_perm = fold_data['n_perm']
#     ignore_adjacency = fold_data['ignore_adjacency']
#     random_state = fold_data['random_state']
#     fold_idx = fold_data['fold_idx']
#     electrodes_per_subject_roi = fold_data['electrodes_per_subject_roi']
#     roi = fold_data['roi']
#     channel_labels = fold_data['channel_labels']
#     freq_labels = fold_data['freq_labels']
#     time_labels = fold_data['time_labels']
#     tfr = fold_data['tfr']
#     cats = fold_data['cats']
    
#     print(f"    Processing fold {fold_idx + 1}")
    
#     # 1. Create training TFR subset
#     train_tfrs = create_train_tfr_subset(
#         subjects_tfr_objects, train_trials, subjects, condition_names
#     )
    
#     # 2. Find significant clusters using only training data
#     train_masks, _ = get_sig_tfr_differences_per_subject(
#         subjects_tfr_objects=train_tfrs,
#         condition_names=condition_names,
#         stat_func=stat_func,
#         p_thresh=p_thresh,
#         n_perm=n_perm,
#         ignore_adjacency=ignore_adjacency,
#         seed=random_state,
#         tails=2
#     )
    
#     # 3. Create feature mask
#     feature_mask = create_feature_mask(
#         train_masks, electrodes_per_subject_roi, roi, subjects,
#         channel_labels, freq_labels, time_labels, tfr
#     )
    
#     n_features = np.sum(feature_mask)
#     print(f"    Using {n_features} features from significant clusters")
    
#     # 4. Extract and clean features
#     X_train, X_test = extract_and_clean_features(train_data, test_data, feature_mask)
    
#     # 5. Balance training data
#     X_train_balanced, y_train_balanced = balance_training_data(
#         X_train, y_train, random_state + fold_idx
#     )
    
#     # 6. Use the Decoder class (PCA-LDA) instead of SVC
#     # The Decoder class uses PcaLdaClassification internally
#     # First parameter is the percentage of variance to retain (0.80 = 80%)
#     decoder = Decoder(cats, 0.80, n_splits=1, n_repeats=1, oversample=False)
    
#     # Fit and predict using the decoder's methods
#     decoder.fit(X_train_balanced, y_train_balanced)
#     preds = decoder.predict(X_test)
    
#     return confusion_matrix(y_test, preds, labels=list(cats.values()))

# def run_cv_cluster_decoding(
#     subjects_tfr_objects,
#     electrodes_per_subject_roi,
#     rois,
#     subjects,
#     condition_names,
#     stat_func,
#     p_thresh,
#     n_perm,
#     ignore_adjacency,
#     n_splits=5,
#     n_repeats=5,
#     random_state=42
# ):
#     """
#     Main function that orchestrates the cross-validated cluster-based decoding.
#     """
#     results_per_roi = {}
#     cats = {name: i for i, name in enumerate(condition_names)}
    
#     for roi in rois:
#         print(f"\n--- Starting Cross-Validation for ROI: {roi} ---")
        
#         # 1. Collect all data for this ROI
#         all_data, trial_labels, condition_labels, channel_labels, freq_labels, time_labels = \
#             collect_roi_data(subjects_tfr_objects, electrodes_per_subject_roi, 
#                            roi, subjects, condition_names)
        
#         if len(all_data) == 0:
#             print(f"  No data found for ROI: {roi}")
#             continue
        
#         # Create LabeledArray
#         roi_data = LabeledArray(
#             all_data,
#             labels=[trial_labels, channel_labels, freq_labels, time_labels]
#         )
        
#         # Create label array for classification
#         y_labels = np.array([cats[cond] for cond in condition_labels])
        
#         # 2. Set up cross-validation
#         skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

#         # Store CMs organized by repeat
#         cms_per_repeat = []
        
#         # Get a sample TFR for channel name lookups
#         sample_tfr = subjects_tfr_objects[subjects[0]][condition_names[0]]

#         for repeat_idx in range(n_repeats):
#             cms_this_repeat = []
#             # different random state for each repeat
#             repeat_random_state = random_state + repeat_idx * 1000
#             skf = StratifiedKFold(n_splits=n_splits, shuffle=True, 
#                                 random_state=repeat_random_state)

#             for fold_idx, (train_indices, test_indices) in enumerate(skf.split(np.zeros(len(y_labels)), y_labels)):
#                 print(f"  --- Running Fold {fold_idx + 1}/{n_splits} ---")

#                 # Create the list of trial labels for indexing
#                 train_label_list = [trial_labels[i] for i in train_indices]
#                 test_label_list = [trial_labels[i] for i in test_indices]
                
#                 # Prepare fold data with corrected, explicit indexing
#                 fold_data = {
#                     'train_trials': train_label_list,
#                     'test_trials': test_label_list,
#                     # Corrected Lines: Wrap the list in a tuple to ensure it's treated as an index for the first dimension.
#                     'train_data': roi_data[(train_label_list,)], 
#                     'test_data': roi_data[(test_label_list,)],
#                     'y_train': y_labels[train_indices],
#                     'y_test': y_labels[test_indices],
#                     'subjects_tfr_objects': subjects_tfr_objects,
#                     'subjects': subjects,
#                     'condition_names': condition_names,
#                     'stat_func': stat_func,
#                     'p_thresh': p_thresh,
#                     'n_perm': n_perm,
#                     'ignore_adjacency': ignore_adjacency,
#                     'random_state': random_state,
#                     'fold_idx': fold_idx,
#                     'electrodes_per_subject_roi': electrodes_per_subject_roi,
#                     'roi': roi,
#                     'channel_labels': channel_labels,
#                     'freq_labels': freq_labels,
#                     'time_labels': time_labels,
#                     'tfr': sample_tfr,
#                     'cats': cats
#                 }

#                 # Run single fold
#                 cm = run_single_cv_fold(fold_data)
#                 cms_this_repeat.append(cm)
#             # sum across folds for this repeat
#             cm_summed = np.sum(cms_this_repeat, axis=0)
#             cms_per_repeat.append(cm_summed)
        
#         # Average across repeats
#         final_cm = np.mean(cms_per_repeat, axis=0)
#         accuracy = np.trace(final_cm) / np.sum(final_cm)
        
#         results_per_roi[roi] = {
#             'confusion_matrix': final_cm,
#             'accuracy': accuracy,
#             'cats': cats,
#             'labeled_array': roi_data,
#             'fold_cms': cms_per_repeat  # Keep individual fold results for analysis
#         }
        
#         print(f"  --> Final Accuracy for {roi}: {accuracy:.3f}")
    
#     return results_per_roi

okay trying again using the approach from my notes 7/24

In [ ]:

def get_subjects_tfr_objects_for_specific_roi(subjects_tfr_objects: Dict, 
                                              electrodes_per_subject_roi: Dict,
                                              roi: str, 
                                              subjects: List[str], 
                                              condition_names: List[str]) -> Dict:
    """
    Filter subjects_tfr_objects to only include electrodes from a specific ROI.
    Maintains the same structure as the input but with filtered channels.
    
    Parameters:
    -----------
    subjects_tfr_objects : dict
        Original structure: {subject: {condition: tfr_object}}
    electrodes_per_subject_roi : dict
        Structure: {roi: {subject: [electrode_names]}}
    roi : str
        The ROI to filter for
    subjects : list
        List of subject IDs
    condition_names : list
        List of condition names
    
    Examples
    --------
    >>> import mne
    >>> import numpy as np
    >>> from typing import Dict, List
    >>> # 1. Create fake data for the example
    >>> ch_names = ['CH1', 'CH2', 'CH3', 'CH4', 'CH5']
    >>> sfreq = 200.0
    >>> info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='seeg')
    >>> n_epochs, n_freqs, n_times = 2, 10, 50
    >>> data = np.random.randn(n_epochs, len(ch_names), n_freqs, n_times)
    >>> freqs = np.arange(10., 20., 1.)
    >>> times = np.linspace(-0.5, 1.0, n_times)
    >>> tfr_mock = mne.time_frequency.EpochsTFR(info, data, times, freqs)
    >>> subjects_tfr_objects_mock = {'sub_1': {'condition_1': tfr_mock, 'condition_2': tfr_mock.copy()}}
    >>> # Define which electrodes belong to which ROI for the subject
    >>> electrodes_per_subject_roi_mock = {
    ...     'roi_1': {'sub_1': ['CH1', 'CH2', 'CH3']},
    ...     'roi_2': {'sub_1': ['CH4', 'CH5']}
    ... }
    >>> # 2. Run the function
    >>> subjects_tfr_objects_for_roi_1 = get_subjects_tfr_objects_for_specific_roi(
    ...     subjects_tfr_objects=subjects_tfr_objects_mock,
    ...     electrodes_per_subject_roi=electrodes_per_subject_roi_mock,
    ...     roi='roi_1',
    ...     subjects=['sub_1'],
    ...     condition_names=['condition_1', 'condition_2']
    ... )
    Subject sub_1, Condition condition_1: Selected 3/5 channels for ROI roi_1
    Subject sub_1, Condition condition_2: Selected 3/5 channels for ROI roi_1
    >>> # 3. Check the result
    >>> print("Original channels:", subjects_tfr_objects_mock['sub_1']['condition_1'].ch_names)
    Original channels: ['CH1', 'CH2', 'CH3', 'CH4', 'CH5']
    >>> print("Filtered channels for roi_1:", subjects_tfr_objects_for_roi_1['sub_1']['condition_1'].ch_names)
    Filtered channels for roi_1: ['CH1', 'CH2', 'CH3']

    Returns:
    --------
    subjects_tfr_objects_for_specific_roi : dict
        Same structure as input: {subject: {condition: tfr_object}}
        but tfr_objects only contain channels from the specified ROI
    """
    subjects_tfr_objects_for_specific_roi = {}
    
    for sub in subjects:
        # Get electrodes for this subject in this ROI
        roi_electrodes = electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        
        # Skip subject if they have no electrodes in this ROI
        if not roi_electrodes:
            print(f"Subject {sub} has no electrodes in ROI {roi}, skipping...")
            continue
            
        subjects_tfr_objects_for_specific_roi[sub] = {}
        
        for cond in condition_names:
            # Get the original TFR object
            tfr = subjects_tfr_objects[sub][cond]
            
            # Find which ROI electrodes are actually in this TFR epochs object
            available_roi_electrodes = [elec for elec in roi_electrodes if elec in tfr.ch_names]
            
            if available_roi_electrodes:
                # Use MNE's pick method to select only ROI channels
                roi_tfr = tfr.copy().pick(available_roi_electrodes)
                subjects_tfr_objects_for_specific_roi[sub][cond] = roi_tfr
                
                print(f"Subject {sub}, Condition {cond}: Selected {len(available_roi_electrodes)}/{len(tfr.ch_names)} channels for ROI {roi}")
            else:
                print(f"Warning: Subject {sub}, Condition {cond} has no channels in ROI {roi}")
                # You could either skip this condition or include an empty TFR
                # Option 1: Skip (don't add to dictionary)
                # Option 2: Add empty TFR (uncomment below)
                # subjects_tfr_objects_for_specific_roi[sub][cond] = tfr.copy().pick([])
    
    return subjects_tfr_objects_for_specific_roi


In [ ]:
# untested
import numpy as np
from sklearn.model_selection import StratifiedKFold
from typing import Dict, List, Tuple, Optional
import mne
from copy import deepcopy
from sklearn.metrics import confusion_matrix

def prepare_data_with_nan_padding(roi_tfr_objects: Dict, 
                                 condition_names: List[str]) -> Tuple[np.ndarray, np.ndarray, Dict]:
    """
    Prepare data matrix with NaN padding to ensure consistent trial counts across subjects.
    Modified to work with multi-channel TFR objects.
    
    Returns:
    --------
    data_matrix : np.ndarray
        Shape: (n_total_trials, n_total_channels, n_freqs, n_times)
    labels : np.ndarray
        Condition labels for each trial
    metadata : dict
        Information about data organization
    """
    # First, find maximum trials per condition across all subjects
    max_trials_per_condition = {cond: 0 for cond in condition_names}
    subject_channel_info = []
    
    for sub, sub_data in roi_tfr_objects.items():
        for cond in condition_names:
            if cond in sub_data:
                tfr = sub_data[cond]
                n_trials = len(tfr)
                max_trials_per_condition[cond] = max(max_trials_per_condition[cond], n_trials)
                # Store info about channels for this subject
                for ch_idx, ch_name in enumerate(tfr.ch_names):
                    subject_channel_info.append((sub, ch_name, ch_idx))
    
    print(f"Maximum trials per condition: {max_trials_per_condition}")
    
    # Get unique subject-channel pairs
    subject_channels = list(set(subject_channel_info))
    n_total_channels = len(subject_channels)
    
    # Get dimensions from first TFR
    sample_tfr = next(iter(next(iter(roi_tfr_objects.values())).values()))
    n_freqs = len(sample_tfr.freqs)
    n_times = len(sample_tfr.times)
    
    # Calculate total trials
    total_trials = sum(max_trials_per_condition.values())
    
    # Initialize data matrix with NaNs
    data_matrix = np.full((total_trials, n_total_channels, n_freqs, n_times), np.nan)
    labels = np.zeros(total_trials, dtype=int)
    
    # Create channel index mapping
    channel_to_idx = {(sub, ch): idx for idx, (sub, ch, _) in enumerate(subject_channels)}
    
    # Fill the matrix
    trial_idx = 0
    trial_metadata = []
    
    for cond_idx, cond in enumerate(condition_names):
        max_trials = max_trials_per_condition[cond]
        
        for trial_num in range(max_trials):
            # For each subject, add this trial if it exists
            for sub in roi_tfr_objects:
                if cond in roi_tfr_objects[sub]:
                    tfr = roi_tfr_objects[sub][cond]
                    if trial_num < len(tfr):
                        # Extract trial data for all channels of this subject
                        trial_data = tfr.data[trial_num, :, :, :]  # (channels, freqs, times)
                        
                        # Place each channel's data in the correct position
                        for ch_idx, ch_name in enumerate(tfr.ch_names):
                            global_ch_idx = channel_to_idx[(sub, ch_name)]
                            data_matrix[trial_idx, global_ch_idx, :, :] = trial_data[ch_idx, :, :]
            
            labels[trial_idx] = cond_idx
            trial_metadata.append({
                'trial_idx': trial_idx,
                'condition': cond,
                'condition_idx': cond_idx,
                'trial_num_in_condition': trial_num
            })
            trial_idx += 1
    
    metadata = {
        'subject_channels': subject_channels,  # List of (subject, channel) tuples
        'channel_to_idx': channel_to_idx,
        'conditions': condition_names,
        'max_trials_per_condition': max_trials_per_condition,
        'trial_metadata': trial_metadata,
        'freqs': sample_tfr.freqs,
        'times': sample_tfr.times,
        'subjects': list(roi_tfr_objects.keys())
    }
    
    return data_matrix, labels, metadata

def split_tfr_objects_by_trials(roi_tfr_objects: Dict, 
                               train_indices: np.ndarray,
                               metadata: Dict) -> Dict:
    """
    Create a subset of TFR objects containing only training trials.
    Modified to work with the simpler structure.
    
    Returns:
    --------
    train_tfr_objects : dict
        Same structure as roi_tfr_objects but with only training trials
    """
    train_tfr_objects = deepcopy(roi_tfr_objects)
    
    # Map train indices to condition/trial pairs
    train_trials_by_condition = {cond: [] for cond in metadata['conditions']}
    
    for idx in train_indices:
        trial_info = metadata['trial_metadata'][idx]
        cond = trial_info['condition']
        trial_num = trial_info['trial_num_in_condition']
        train_trials_by_condition[cond].append(trial_num)
    
    # Filter TFR objects to only include training trials
    for sub in train_tfr_objects:
        for cond in metadata['conditions']:
            if cond in train_tfr_objects[sub]:
                tfr = train_tfr_objects[sub][cond]
                valid_trials = [t for t in train_trials_by_condition[cond] if t < len(tfr)]
                if valid_trials:
                    train_tfr_objects[sub][cond] = tfr[valid_trials]
                else:
                    # Create empty TFR if no valid trials
                    train_tfr_objects[sub][cond] = tfr[:0]
    
    return train_tfr_objects


def compute_tfr_masks_from_training(train_tfr_objects: Dict,
                                   condition_names: List[str],
                                   stat_func: callable,
                                   p_thresh: float,
                                   n_perm: int,
                                   metadata: Dict,
                                   ignore_adjacency: int = 1,
                                   seed: int = 42,
                                   tails: int = 2) -> Dict:
    """
    Compute significant TFR clusters using only training data.
    Modified to handle multi-channel TFR objects.
    
    Returns:
    --------
    channel_masks : dict
        {(subject, channel): mask_array} where mask is (n_freqs, n_times)
    """
    channel_masks = {}
    
    # Process each subject's data
    for sub in metadata['subjects']:
        if sub not in train_tfr_objects:
            continue
        
        # Get TFRs for both conditions
        tfrs_for_comparison = []
        for cond in condition_names[:2]:  # Assuming binary comparison
            if cond in train_tfr_objects[sub]:
                tfr = train_tfr_objects[sub][cond]
                if len(tfr) > 0:
                    tfrs_for_comparison.append(tfr)
        
        if len(tfrs_for_comparison) == 2:
            # Run statistical test on multi-channel data
            mask, _ = get_sig_tfr_differences(
                tfr_data_cond1=tfrs_for_comparison[0],
                tfr_data_cond2=tfrs_for_comparison[1],
                stat_func=stat_func,
                p_thresh=p_thresh,
                n_perm=n_perm,
                axis=0,  # trials axis
                ignore_adjacency=ignore_adjacency,
                seed=seed,
                tails=tails
            )
            # mask shape: (n_channels, n_freqs, n_times)
            
            # Store mask for each channel
            tfr = tfrs_for_comparison[0]  # Use to get channel names
            for ch_idx, ch_name in enumerate(tfr.ch_names):
                if ch_idx < mask.shape[0]:
                    channel_masks[(sub, ch_name)] = mask[ch_idx, :, :]
                else:
                    # If mask doesn't have enough channels, use zeros
                    channel_masks[(sub, ch_name)] = np.zeros((len(metadata['freqs']), 
                                                              len(metadata['times'])), dtype=bool)
        else:
            # No data for comparison, create zero masks
            if sub in train_tfr_objects:
                for cond in train_tfr_objects[sub]:
                    if cond in train_tfr_objects[sub]:
                        tfr = train_tfr_objects[sub][cond]
                        for ch_name in tfr.ch_names:
                            channel_masks[(sub, ch_name)] = np.zeros((len(metadata['freqs']), 
                                                                      len(metadata['times'])), dtype=bool)
    
    return channel_masks


def apply_masks_and_flatten(data_matrix: np.ndarray, 
                           channel_masks: Dict,
                           metadata: Dict) -> np.ndarray:
    """
    Apply TFR masks to data and flatten to create feature matrix.
    
    Returns:
    --------
    feature_matrix : np.ndarray
        Shape: (n_trials, n_features) where features are concatenated across channels
    """
    n_trials = data_matrix.shape[0]
    feature_vectors = []
    
    # For each channel, apply mask and flatten
    for (sub, ch_name), global_ch_idx in metadata['channel_to_idx'].items():
        channel_data = data_matrix[:, global_ch_idx, :, :]  # (n_trials, n_freqs, n_times)
        
        if (sub, ch_name) in channel_masks:
            mask = channel_masks[(sub, ch_name)]
            
            # Apply mask and flatten for each trial
            masked_features = []
            for trial in range(n_trials):
                trial_data = channel_data[trial, :, :]
                masked_data = trial_data[mask]  # 1D vector of masked features
                masked_features.append(masked_data)
            
            masked_features = np.array(masked_features)  # (n_trials, n_masked_features)
        else:
            # If no mask, use all features
            masked_features = channel_data.reshape(n_trials, -1)
        
        feature_vectors.append(masked_features)
    
    # Concatenate features across channels
    feature_matrix = np.concatenate(feature_vectors, axis=1)
    
    return feature_matrix

def run_tfr_cluster_decoding_pipeline(subjects_tfr_objects: Dict,
                                     electrodes_per_subject_roi: Dict,
                                     roi: str,
                                     subjects: List[str],
                                     condition_names: List[str],
                                     stat_func: callable,
                                     p_thresh: float = 0.05,
                                     n_perm: int = 100,
                                     n_splits: int = 5,
                                     n_repeats: int = 10,
                                     balance_method: str = 'subsample',
                                     n_bootstraps: int = 10,
                                     random_state: int = 42,
                                     ignore_adjacency: int = 1,
                                     seed: int = 42,
                                     tails: int = 2) -> Dict:
    """
    Main pipeline function that orchestrates the entire decoding process.
    Modified to use get_subjects_tfr_objects_for_specific_roi.
    """
    print(f"\n=== Running TFR Cluster Decoding for ROI: {roi} ===")
    
    # Step 1: Filter TFR objects for specific ROI
    roi_tfr_objects = get_subjects_tfr_objects_for_specific_roi(
        subjects_tfr_objects, electrodes_per_subject_roi, 
        roi, subjects, condition_names
    )
    
    if not roi_tfr_objects:
        print(f"No data found for ROI: {roi}")
        return None
    
    # Step 2: Prepare data with NaN padding (BEFORE splitting)
    data_matrix, labels, metadata = prepare_data_with_nan_padding(
        roi_tfr_objects, condition_names
    )
    
    print(f"Data matrix shape: {data_matrix.shape}")
    print(f"Number of channels: {len(metadata['subject_channels'])}")
    print(f"Number of NaN trials: {np.isnan(data_matrix).all(axis=(1,2,3)).sum()}")
    
    # Step 3: Set up cross-validation
    cats = {name: i for i, name in enumerate(condition_names)}
    
    # Storage for results
    all_cms = []
    
    # Step 4: Repeat the entire process
    for repeat in range(n_repeats):
        repeat_seed = random_state + repeat * 1000
        skf_repeat = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=repeat_seed)
        
        fold_cms = []
        
        # Step 5: Cross-validation folds
        for fold_idx, (train_indices, test_indices) in enumerate(skf_repeat.split(np.zeros(len(labels)), labels)):
            print(f"\n  Repeat {repeat+1}/{n_repeats}, Fold {fold_idx+1}/{n_splits}")
            
            # Step 6: Split data
            X_train_raw = data_matrix[train_indices]
            X_test_raw = data_matrix[test_indices]
            y_train = labels[train_indices]
            y_test = labels[test_indices]
            
            # Step 7: Create training-only TFR objects for computing masks
            train_tfr_objects = split_tfr_objects_by_trials(
                roi_tfr_objects, train_indices, metadata
            )
            
            # Step 8: Compute masks using ONLY training data
            channel_masks = compute_tfr_masks_from_training(
                train_tfr_objects, condition_names, 
                stat_func, p_thresh, n_perm, metadata,
                ignore_adjacency, seed, tails
            )
            
            n_sig_channels = sum(1 for mask in channel_masks.values() if mask.any())
            print(f"    Found significant clusters in {n_sig_channels}/{len(metadata['subject_channels'])} channels")
            
            # Step 9: Apply masks to both train and test data
            X_train = apply_masks_and_flatten(X_train_raw, channel_masks, metadata)
            X_test = apply_masks_and_flatten(X_test_raw, channel_masks, metadata)
            
            print(f"    Feature matrix shape: {X_train.shape}")
            
            # Step 10: Balance and decode
            cm = balance_and_decode(
                X_train, y_train, X_test, y_test,
                Decoder, cats, balance_method, 
                n_bootstraps, repeat_seed + fold_idx
            )
            
            fold_cms.append(cm)
        
        # Sum across folds for this repeat
        repeat_cm = np.sum(fold_cms, axis=0)
        all_cms.append(repeat_cm)
    
    # Average across repeats
    final_cm = np.mean(all_cms, axis=0)
    accuracy = np.trace(final_cm) / np.sum(final_cm)
    
    results = {
        'confusion_matrix': final_cm,
        'accuracy': accuracy,
        'cats': cats,
        'metadata': metadata,
        'all_repeat_cms': all_cms
    }
    
    print(f"\n=== Final accuracy for {roi}: {accuracy:.3f} ===")
    
    return results

plotting functions

In [ ]:
def plot_cv_confusion_matrices(cv_results, save_dir, conditions_save_name):
    """
    Plot confusion matrices for cross-validated cluster-based decoding results.
    
    Parameters:
    -----------
    cv_results : dict
        Results from run_cv_cluster_decoding containing confusion matrices per ROI
    save_dir : str
        Directory to save plots
    conditions_save_name : str
        Name identifier for the conditions being compared
    """
    for roi, results in cv_results.items():
        # Get the final aggregated confusion matrix
        cm = results['confusion_matrix']
        cats = results['cats']
        accuracy = results['accuracy']
        
        # Also get individual fold confusion matrices
        fold_cms = results.get('fold_cms', [])
        
        # 1. Plot the aggregated confusion matrix
        plot_single_confusion_matrix(
            cm, cats, accuracy, roi, 
            f'{roi}_CV_cluster_decoded_final_cm_{conditions_save_name}',
            save_dir,
            title_suffix='Final Aggregated'
        )
        
        # 2. Plot individual fold confusion matrices in a grid
        if fold_cms:
            plot_fold_confusion_matrices(
                fold_cms, cats, roi,
                f'{roi}_CV_cluster_decoded_folds_{conditions_save_name}',
                save_dir
            )
        
        # 3. Plot normalized confusion matrix
        plot_normalized_confusion_matrix(
            cm, cats, accuracy, roi,
            f'{roi}_CV_cluster_decoded_normalized_{conditions_save_name}',
            save_dir
        )


def plot_single_confusion_matrix(cm, cats, accuracy, roi, filename, save_dir, 
                                title_suffix='', normalize=False):
    """
    Plot a single confusion matrix with customizable normalization.
    """
    # Normalize if requested
    if normalize:
        cm_plot = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_plot = np.nan_to_num(cm_plot)  # Handle division by zero
    else:
        cm_plot = cm
    
    display_labels = list(cats.keys())
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Create the display
    if normalize:
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_plot, display_labels=display_labels)
        im = disp.plot(ax=ax, cmap='Blues', values_format='.2f')
        im.im_.set_clim(0, 1)
    else:
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_plot, display_labels=display_labels)
        im = disp.plot(ax=ax, cmap='Blues', values_format='d')
    
    # Add title
    title = f'ROI: {roi} - Accuracy: {accuracy:.3f}'
    if title_suffix:
        title += f' ({title_suffix})'
    ax.set_title(title, fontsize=14, pad=20)
    
    # Enhance the plot
    ax.set_xlabel('Predicted Label', fontsize=12)
    ax.set_ylabel('True Label', fontsize=12)
    
    # Add grid for better readability
    ax.set_xticks(np.arange(len(display_labels)) - 0.5, minor=True)
    ax.set_yticks(np.arange(len(display_labels)) - 0.5, minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f'{filename}.png'), dpi=300, bbox_inches='tight')
    plt.close()


def plot_fold_confusion_matrices(fold_cms, cats, roi, filename, save_dir):
    """
    Plot all fold confusion matrices in a grid layout.
    """
    n_folds = len(fold_cms)
    n_cols = min(3, n_folds)
    n_rows = (n_folds + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    if n_folds == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    display_labels = list(cats.keys())
    
    for idx, cm in enumerate(fold_cms):
        ax = axes[idx]
        
        # Normalize the confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_normalized = np.nan_to_num(cm_normalized)
        
        # Calculate fold accuracy
        fold_accuracy = np.trace(cm) / np.sum(cm)
        
        # Plot
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, 
                                    display_labels=display_labels)
        im = disp.plot(ax=ax, cmap='Blues', values_format='.2f', colorbar=False)
        im.im_.set_clim(0, 1)
        
        ax.set_title(f'Fold {idx+1} (Acc: {fold_accuracy:.3f})', fontsize=12)
        ax.set_xlabel('')
        ax.set_ylabel('')
    
    # Remove unused subplots
    for idx in range(n_folds, len(axes)):
        fig.delaxes(axes[idx])
    
    # Add overall title
    fig.suptitle(f'ROI: {roi} - Individual Fold Confusion Matrices', fontsize=16)
    
    # Add shared labels
    fig.text(0.5, 0.02, 'Predicted Label', ha='center', fontsize=14)
    fig.text(0.02, 0.5, 'True Label', va='center', rotation='vertical', fontsize=14)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f'{filename}.png'), dpi=300, bbox_inches='tight')
    plt.close()


def plot_normalized_confusion_matrix(cm, cats, accuracy, roi, filename, save_dir):
    """
    Plot a normalized confusion matrix showing percentages.
    """
    # Calculate different normalizations
    cm_row_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm_row_norm = np.nan_to_num(cm_row_norm)
    
    display_labels = list(cats.keys())
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Row-normalized (recall)
    disp1 = ConfusionMatrixDisplay(confusion_matrix=cm_row_norm, 
                                  display_labels=display_labels)
    im1 = disp1.plot(ax=ax1, cmap='Blues', values_format='.2%')
    im1.im_.set_clim(0, 1)
    ax1.set_title(f'Row-Normalized (Recall)\nAccuracy: {accuracy:.3f}', fontsize=14)
    
    # Plot 2: Raw counts with percentages
    total_samples = np.sum(cm)
    cm_percentage = cm / total_samples
    
    # Create custom annotation
    im2 = ax2.imshow(cm_percentage, cmap='Blues', aspect='auto')
    
    # Add text annotations showing both count and percentage
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            count = cm[i, j]
            pct = cm_percentage[i, j] * 100
            text = f'{int(count)}\n({pct:.1f}%)'
            color = 'white' if cm_percentage[i, j] > 0.5 else 'black'
            ax2.text(j, i, text, ha='center', va='center', color=color, fontsize=10)
    
    ax2.set_xticks(range(len(display_labels)))
    ax2.set_yticks(range(len(display_labels)))
    ax2.set_xticklabels(display_labels)
    ax2.set_yticklabels(display_labels)
    ax2.set_xlabel('Predicted Label')
    ax2.set_ylabel('True Label')
    ax2.set_title(f'Raw Counts with Percentages\nTotal Samples: {int(total_samples)}', fontsize=14)
    
    # Add colorbar
    cbar = plt.colorbar(im2, ax=ax2)
    cbar.set_label('Proportion of Total', rotation=270, labelpad=20)
    
    fig.suptitle(f'ROI: {roi} - Detailed Confusion Matrix Analysis', fontsize=16)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f'{filename}.png'), dpi=300, bbox_inches='tight')
    plt.close()


def plot_accuracy_comparison_across_rois(cv_results, save_dir, conditions_save_name):
    """
    Create a bar plot comparing accuracies across ROIs.
    """
    rois = []
    accuracies = []
    
    for roi, results in cv_results.items():
        rois.append(roi)
        accuracies.append(results['accuracy'])
    
    # Create bar plot
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(rois, accuracies, color='steelblue', alpha=0.8)
    
    # Add value labels on bars
    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{acc:.3f}', ha='center', va='bottom', fontsize=12)
    
    # Add chance level line (assuming binary classification)
    n_classes = len(list(cv_results.values())[0]['cats'])
    chance_level = 1.0 / n_classes
    ax.axhline(y=chance_level, color='red', linestyle='--', alpha=0.7, 
               label=f'Chance Level ({chance_level:.3f})')
    
    ax.set_xlabel('ROI', fontsize=14)
    ax.set_ylabel('Accuracy', fontsize=14)
    ax.set_title(f'Decoding Accuracy Across ROIs\n{conditions_save_name}', fontsize=16)
    ax.set_ylim(0, 1.1)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    filename = f'accuracy_comparison_across_rois_{conditions_save_name}.png'
    plt.savefig(os.path.join(save_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()


def save_cv_results_summary(cv_results, save_dir, conditions_save_name):
    """
    Save a text summary of the cross-validation results.
    """
    summary_file = os.path.join(save_dir, f'cv_results_summary_{conditions_save_name}.txt')
    
    with open(summary_file, 'w') as f:
        f.write(f"Cross-Validation Cluster-Based Decoding Results\n")
        f.write(f"Conditions: {conditions_save_name}\n")
        f.write("="*60 + "\n\n")
        
        for roi, results in cv_results.items():
            f.write(f"ROI: {roi}\n")
            f.write(f"Final Accuracy: {results['accuracy']:.4f}\n")
            f.write(f"Number of folds: {len(results.get('fold_cms', []))}\n")
            
            # Calculate per-class metrics
            cm = results['confusion_matrix']
            cats = results['cats']
            
            f.write("\nPer-class metrics:\n")
            for i, (cat_name, cat_idx) in enumerate(cats.items()):
                if i < cm.shape[0]:  # Ensure we don't go out of bounds
                    tp = cm[i, i]
                    fp = np.sum(cm[:, i]) - tp
                    fn = np.sum(cm[i, :]) - tp
                    
                    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
                    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
                    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
                    
                    f.write(f"  {cat_name}:\n")
                    f.write(f"    Precision: {precision:.4f}\n")
                    f.write(f"    Recall: {recall:.4f}\n")
                    f.write(f"    F1-score: {f1:.4f}\n")
            
            f.write("\nConfusion Matrix:\n")
            f.write(str(cm) + "\n")
            f.write("\n" + "="*60 + "\n\n")


# Enhanced main execution with plotting
def run_analysis_with_plots(subjects_tfr_objects, all_electrodes_per_subject_roi, 
                           rois, subjects, condition_names, stat_func, p_thresh, 
                           n_perm, ignore_adjacency, n_splits, n_repeats, random_state,save_dir, conditions_save_name):
    """
    Run the complete analysis pipeline with enhanced plotting.
    """
    # Ensure save directory exists
    os.makedirs(save_dir, exist_ok=True)
    
    # Run the cross-validated analysis
    print("Running cross-validated cluster-based decoding...")
    cv_results = run_cv_cluster_decoding(
        subjects_tfr_objects=subjects_tfr_objects,
        electrodes_per_subject_roi=all_electrodes_per_subject_roi,
        rois=rois,
        subjects=subjects,
        condition_names=condition_names,
        stat_func=stat_func,
        p_thresh=p_thresh,
        n_perm=n_perm,
        ignore_adjacency=ignore_adjacency,
        n_splits=n_splits,
        n_repeats=n_repeats,
        random_state=random_state
    )
    
    # Generate all plots
    print("Generating confusion matrix plots...")
    plot_cv_confusion_matrices(cv_results, save_dir, conditions_save_name)
    
    print("Generating accuracy comparison plot...")
    plot_accuracy_comparison_across_rois(cv_results, save_dir, conditions_save_name)
    
    print("Saving results summary...")
    save_cv_results_summary(cv_results, save_dir, conditions_save_name)
    
    print("Analysis complete!")
    return cv_results

In [ ]:
# --- HOW TO RUN THE NEW PIPELINE ---

# Make sure your save directory is defined
save_dir = os.path.join(layout.root, 'derivatives', 'decoding', 'james_sun_cluster_decoding', conditions_save_name)
os.makedirs(save_dir, exist_ok=True)

# Run the enhanced analysis with all plots
cv_results = run_analysis_with_plots(
    subjects_tfr_objects=subjects_tfr_objects,
    all_electrodes_per_subject_roi=all_electrodes_per_subject_roi,
    rois=rois,
    subjects=subjects,
    condition_names=condition_names,
    stat_func=stat_func,
    p_thresh=p_thresh,
    n_perm=n_perm,
    ignore_adjacency=ignore_adjacency,
    n_splits=n_splits,
    n_repeats=n_repeats,
    random_state=random_state,
    save_dir=save_dir,
    conditions_save_name=conditions_save_name
)

with open(os.path.join(save_dir, f'cv_results_{conditions_save_name}.pkl'), 'wb') as f:
    pickle.dump(cv_results, f)

#### 3. Train a decoder on just the significant time-frequency clusters, test on the test trials

#### 4. Repeat but with new test trials (cross-validate)

#### 5. Figure out a way of plotting this. This is univariate approach, make things modular so I can do multivariate later once I figure that out.

In [4]:
doctest.testmod(verbose=True)

Trying:
    import mne
Expecting nothing
ok
Trying:
    import numpy as np
Expecting nothing
ok
Trying:
    from typing import Dict, List
Expecting nothing
ok
Trying:
    ch_names = ['CH1', 'CH2', 'CH3', 'CH4', 'CH5']
Expecting nothing
ok
Trying:
    sfreq = 200.0
Expecting nothing
ok
Trying:
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='seeg')
Expecting nothing
ok
Trying:
    n_epochs, n_freqs, n_times = 2, 10, 50
Expecting nothing
ok
Trying:
    data = np.random.randn(n_epochs, len(ch_names), n_freqs, n_times)
Expecting nothing
ok
Trying:
    freqs = np.arange(10., 20., 1.)
Expecting nothing
ok
Trying:
    times = np.linspace(-0.5, 1.0, n_times)
Expecting nothing
ok
Trying:
    tfr_mock = mne.time_frequency.EpochsTFR(info, data, times, freqs)
Expecting nothing
ok
Trying:
    subjects_tfr_objects_mock = {'sub_1': {'condition_1': tfr_mock, 'condition_2': tfr_mock.copy()}}
Expecting nothing
ok
Trying:
    electrodes_per_subject_roi_mock = {
        'roi_1': {'sub_

TestResults(failed=1, attempted=16)